## 0. System Information Beforehand

In [3]:
uname -a

Linux neptune 6.1.0-9-amd64 #1 SMP PREEMPT_DYNAMIC Debian 6.1.27-1 (2023-05-08) x86_64 GNU/Linux


In [4]:
lsb_release -a

No LSB modules are available.
Distributor ID:	Debian
Description:	Debian GNU/Linux 12 (bookworm)
Release:	12
Codename:	bookworm


Install additional packages before building a kernel:

In [5]:
sudo apt-get install -y --no-install-recommends fakeroot \
                                                build-essential \
                                                ncurses-dev \
                                                xz-utils \
                                                libssl-dev \
                                                bc \
                                                flex \
                                                libelf-dev \
                                                bison

Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree... Done
Reading state information... 0% Reading state information... 0%Reading state information... Done
Note, selecting 'libncurses-dev' instead of 'ncurses-dev'
xz-utils is already the newest version (5.4.1-0.2).
bc is already the newest version (1.07.1-3+b1).
bc set to manually installed.
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu dpkg-dev g++ g++-12 gcc
  gcc-12 libasan8 libbinutils libc-dev-bin libc6-dev libcc1-0 libcrypt-dev
  libctf-nobfd0 libctf0 libdpkg-perl libfakeroot libgcc-12-dev libgprofng0
  libitm1 liblsan0 libnsl-dev libstdc++-12-dev libtirpc-dev libtsan2 libubsan1
  linux-libc-dev m4 make patch rpcsvc-proto zlib1g-dev
Suggested packages:
  binutils-doc bison-doc debian-keyring 

## 1. Get Linux Kernel Source Tree

Download package from the linux kernel archives website:

In [6]:
curl -o /tmp/linux-6.1.38.tar.xz "https://cdn.kernel.org/pub/linux/kernel/v6.x/linux-6.1.38.tar.xz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0  4595k      0  0:00:28  0:00:28 --:--:-- 5002k


Extract source files:

In [7]:
btrfs subvolume create $HOME/srchub/linux-6.1.38

Create subvolume '/home/clymber/srchub/linux-6.1.38'


In [8]:
tar -xf /tmp/linux-6.1.38.tar.xz -C $HOME/srchub/

Let's keep the name style for btrfs subvolume to start with '@' sign:

In [9]:
mv $HOME/srchub/linux-6.1.38 "$HOME/srchub/@linux-6.1.38"

Let's work on a snapshot copy to keep the original source code unchanged(optional, btrfs only):

In [10]:
btrfs subvolume snapshot "$HOME/srchub/@linux-6.1.38" "$HOME/studio/@linux-6.1.38"

Create a snapshot of '/home/clymber/srchub/@linux-6.1.38' in '/home/clymber/studio/@linux-6.1.38'


## 2. Configure Kernel

In [1]:
cd $HOME/studio/@linux-6.1.38

For this time, we're going to copy the existed configuration file from current system to simplify the work. The current system kernel configuration  usually locates at `/boot/config-<kernel-release>`.

In [12]:
cp -v /boot/config-$(uname -r) .config

'/boot/config-6.1.0-9-amd64' -> '.config'


We can use the `scripts/config` within the source tree to check/change options for compiling the kernel.

In [13]:
opts="BPF HAVE_EBPF_JIT BPF_SYSCALL BPF_JIT IKHEADERS NET_CLS_BPF NET_ACT_BPF \
      BPF_EVENTS TRUSTED_KEYS REVOCATION_KEYS"
for opt in $opts; do
    echo "CONFIG_${opt}="$(scripts/config --state "CONFIG_${opt}")
done

CONFIG_BPF=y
CONFIG_HAVE_EBPF_JIT=y
CONFIG_BPF_SYSCALL=y
CONFIG_BPF_JIT=y
CONFIG_IKHEADERS=n
CONFIG_NET_CLS_BPF=m
CONFIG_NET_ACT_BPF=m
CONFIG_BPF_EVENTS=y
CONFIG_TRUSTED_KEYS=n
CONFIG_REVOCATION_KEYS=undef


In [14]:
scripts/config --enable "CONFIG_IKHEADERS"

If you are compiling the kernel on Ubuntu, you may receive the following error that interrupts the building process: `No rule to make target 'debian/canonical-certs.pem`, disable the conflicting security certificates by executing the two commands below:

In [15]:
# we don't need this since we're working on a debian
# scripts/config --disable "CONFIG_TRUSTED_KEYS"
# scripts/config --disable "CONFIG_REVOCATION_KEYS"

After making changes to your configuration file, or when using an existing configuration file on a new kernel tree, you can validate and update the configuration: `$ make oldconfig`. You should always run this before building a kernel.

In [16]:
# execute it in the system terminal
# make oldconfig

## 3. Compling

please DO NOT execute the following command in jupyter notebook if not necessary, because it will consume large amount of memory and slow down the system. Do it with the system shell terminal to have better performance.

In [3]:
make -j6

  DESCEND objtool
  DESCEND bpf/resolve_btfids
  CALL    scripts/checksyscalls.sh
  CHK     kernel/kheaders_data.tar.xz
  UPD     include/generated/utsversion.h
  CC      init/version-timestamp.o
BTF: .tmp_vmlinux.btf: pahole (pahole) is not available
Failed to generate BTF for vmlinux
Try to disable CONFIG_DEBUG_INFO_BTF
make[1]: *** [scripts/Makefile.vmlinux:34: vmlinux] Error 1
make: *** [Makefile:1255: vmlinux] Error 2
make: *** Waiting for unfinished jobs....


: 2

In [5]:
sudo apt-get install -y --no-install-recommends dwarves

Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree... Done
Reading state information... 0% Reading state information... 0%Reading state information... Done
The following additional packages will be installed:
  pahole
The following NEW packages will be installed:
  dwarves pahole
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 239 kB of archives.
After this operation, 1,546 kB of additional disk space will be used.
Get:1 http://mirrors.ustc.edu.cn/debian bookworm/main amd64 pahole amd64 1.24-4.1 [229 kB]
Get:2 http://mirrors.ustc.edu.cn/debian bookworm/main amd64 dwarves all 1.24-4.1 [10.2 kB]
Fetched 239 kB in 1s (190 kB/s)
Selecting previously unselected package pahole.
(Reading database ... 146161 files and directories currently installed.)
Preparing to unpack .../pahole_1.24-4

In [6]:
make -j8

  DESCEND objtool
  DESCEND bpf/resolve_btfids
  CALL    scripts/checksyscalls.sh
  CHK     kernel/kheaders_data.tar.xz
  UPD     include/generated/utsversion.h
  CC      init/version-timestamp.o
  LD      .tmp_vmlinux.btf
  BTF     .btf.vmlinux.bin.o
  LD      .tmp_vmlinux.kallsyms1
  NM      .tmp_vmlinux.kallsyms1.syms
  KSYMS   .tmp_vmlinux.kallsyms1.S
  AS      .tmp_vmlinux.kallsyms1.S
  LD      .tmp_vmlinux.kallsyms2
  NM      .tmp_vmlinux.kallsyms2.syms
  KSYMS   .tmp_vmlinux.kallsyms2.S
  AS      .tmp_vmlinux.kallsyms2.S
  LD      vmlinux
  BTFIDS  vmlinux
  NM      System.map
  SORTTAB vmlinux
  CC      arch/x86/boot/a20.o
  AS      arch/x86/boot/bioscall.o
  CC      arch/x86/boot/cmdline.o
  AS      arch/x86/boot/copy.o
  HOSTCC  arch/x86/boot/mkcpustr
  CC      arch/x86/boot/cpuflags.o
  CC      arch/x86/boot/cpucheck.o
  CC      arch/x86/boot/early_serial_console.o
  CC      arch/x86/boot/edd.o
  CC      arch/x86/boot/main.o
  CC      arch/x86/boot/memory.o
  CC      arch/x8

## 4. Install the required modules

In [7]:
sudo make -j8 modules_install

  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/aegis128-aesni.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/aesni-intel.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/blowfish-x86_64.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/camellia-aesni-avx-x86_64.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/camellia-aesni-avx2.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/camellia-x86_64.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/cast5-avx-x86_64.ko
  INSTALL /lib/modules/6.1.38/kernel/arch/x86/crypto/cast6-avx-x86_64.ko
  SIGN    /lib/modules/6.1.38/kernel/arch/x86/crypto/aegis128-aesni.ko
  SIGN    /lib/modules/6.1.38/kernel/arch/x86/crypto/aesni-intel.ko
  SIGN    /lib/modules/6.1.38/kernel/arch/x86/crypto/blowfish-x86_64.ko
  SIGN    /lib/modules/6.1.38/kernel/arch/x86/crypto/camellia-aesni-avx2.ko
  SIGN    /lib/modules/6.1.38/kernel/arch/x86/crypto/camellia-x86_64.ko
  SIGN    /lib/modules/6.1.38/kernel/arch/x86/crypto/c

In [8]:
du -sh /lib/modules/*

391M	/lib/modules/6.1.0-9-amd64
4.7G	/lib/modules/6.1.38


## 5. Install the kernel

In [9]:
sudo make install

  INSTALL /boot
run-parts: executing /etc/kernel/postinst.d/initramfs-tools 6.1.38 /boot/vmlinuz-6.1.38
update-initramfs: Generating /boot/initrd.img-6.1.38
run-parts: executing /etc/kernel/postinst.d/zz-update-grub 6.1.38 /boot/vmlinuz-6.1.38
Generating grub configuration file ...
Found background image: /usr/share/images/desktop-base/desktop-grub.png
Found linux image: /boot/vmlinuz-6.1.38
Found initrd image: /boot/initrd.img-6.1.38
Found linux image: /boot/vmlinuz-6.1.0-9-amd64
Found initrd image: /boot/initrd.img-6.1.0-9-amd64
Its output will be used to detect bootable binaries on them and create new boot entries.
Found Windows Boot Manager on /dev/nvme0n1p1@/EFI/Microsoft/Boot/bootmgfw.efi
Found Ubuntu 22.04.1 LTS (22.04) on /dev/nvme0n1p8
Adding boot menu entry for UEFI Firmware Settings ...
done


## 5. Update the Bootloader (Optional)

The GRUB bootloader is the first program that runs when the system powers on.  
The make install command performs this process automatically, but you can also do it manually.

### 1. <a id="update-initramfs">Update the initramfs to the installed kernel version:</a>

In [2]:
sudo update-initramfs -c -k 6.1.38

update-initramfs: Generating /boot/initrd.img-6.1.38


### 2. Update the GRUB bootloader with this command:

In [3]:
sudo update-grub

Generating grub configuration file ...
Found background image: /usr/share/images/desktop-base/desktop-grub.png
Found linux image: /boot/vmlinuz-6.1.38
Found initrd image: /boot/initrd.img-6.1.38
Found linux image: /boot/vmlinuz-6.1.0-9-amd64
Found initrd image: /boot/initrd.img-6.1.0-9-amd64
Its output will be used to detect bootable binaries on them and create new boot entries.
Found Windows Boot Manager on /dev/nvme0n1p1@/EFI/Microsoft/Boot/bootmgfw.efi
Found Ubuntu 22.04.1 LTS (22.04) on /dev/nvme0n1p8
Adding boot menu entry for UEFI Firmware Settings ...
done


# 6. Reboot and Verify Kernel Version

After completed the steps above, rebooted the machine.  
Unfortunately, the system crashed during its booting process, and output:
```text
Booting 'Debian GNU/Linux'

Loading Linux 6.1.38 ...
Loading initial ramdisk ...
error: out of memory.

Press any key to continue ...
```

I found an answer [here](https://unix.stackexchange.com/questions/698890/out-of-memory-on-loading-initial-ramdisk-after-kernel-upgrade-4-15-to-4-19-o). In short, the problem is that the initrd image is too large:

In [1]:
ls -lh /boot/initrd*

-rw-r--r-- 1 root root  58M Jun 13 14:46 /boot/initrd.img-6.1.0-9-amd64
-rw-r--r-- 1 root root 437M Jul 13 21:34 /boot/initrd.img-6.1.38


It turned out that the newly compiled initrd img is almost 10 times larger the the original one. The _initrd.img-6.1.38_ is generated by [update-initramfs](#update-initramfs) command, and the command's confurations are store in */etc/initramfs-tools/initramfs.conf*

In [11]:
awk '/^# MODULES/,/^MODULES=/' "/etc/initramfs-tools/initramfs.conf"

# MODULES: [ most | netboot | dep | list ]
#
# most - Add most filesystem and all harddrive drivers.
#
# dep - Try and guess which modules to load.
#
# netboot - Add the base modules, network modules, but skip block devices.
#
# list - Only include modules from the 'additional modules' list
#

MODULES=most


So, set **MODULES=dep**:

In [14]:
sudo sed -i 's/^\(MODULES=\).*$/\1dep/' "/etc/initramfs-tools/initramfs.conf"

Coming up next, let's re-generate the initrd.img:

In [15]:
sudo update-initramfs -c -k 6.1.38

update-initramfs: Generating /boot/initrd.img-6.1.38


In [16]:
ls -lh /boot/initrd.img*

-rw-r--r-- 1 root root 58M Jun 13 14:46 /boot/initrd.img-6.1.0-9-amd64
-rw-r--r-- 1 root root 65M Jul 14 12:28 /boot/initrd.img-6.1.38


Note: Another way around to reduce the _initrd_ image size is strip the debug info in the ko file:
```sh
find /lib/modules/$(uname -r)/ -iname "*.ko" -exec strip --strip-unneeded {} \;
```
or, strip the module files during the kernel installation procedure:
```sh
make INSTALL_MOD_STRIP=1 modules_install && make install
```

# Reboot and Verify Kernel Version Again

In [ ]:
# sudo reboot

In [1]:
uname -r

6.1.38
